In [79]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [19]:
def load_data():
    data = pd.read_csv("tweets.csv")[:500]
    return data

In [24]:
df2 = load_data()
df2

/var/folders/g9/hkd92dbd5zn6plc65t5br2p40000gn/T/ipykernel_1500/2857418959.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("tweets.csv")[:500]


,user_name,text,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,hashtags,source
0,Bohmle,"#GPT4 for FREE. \nNo its not a clickbait, @Qol...",Carkingga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,AI enthusiast,2019-07-03 03:44:41+00:00,60.0,349.0,611,False,2023-05-17 18:11:12+00:00,"['GPT4', 'ChatGPT4']",Twitter Web App,NaN,NaN
2,Dan Bruno AI,ChatGPT Thinks These 5 Crypto Coins Will Explo...,"Manchester, NH","The latest in #ChatGPT, #BARD, #Bing, and othe...",2021-05-19 01:19:32+00:00,470.0,157.0,5185,False,2023-05-17 18:11:03+00:00,"['chatgpt', 'AI', 'openAI']",dlvr.it
3,Georgiana Comsa,New: @JWVance's post about 5 #startups (includ...,Palo Alto,"Founder of Silicon Valley PR, award-winning PR...",2008-12-24 09:32:23+00:00,3864.0,1883.0,2415,False,2023-05-17 18:10:25+00:00,"['startups', 'startup50']",Twitter Web App
4,Bitone Great,🚨Get Out!🚨\n💰#Binance Spot💰\n⬇ Recommendation:...,Hong Kong,#ChatGPT (AI) powered Free Trading Signal! \nL...,2022-11-21 04:42:18+00:00,1517.0,506.0,64,False,2023-05-17 18:09:39+00:00,"['Binance', 'Short', 'GHSTUSDT']",rsi1
...,...,...,...,...,...,...,...,...,...,...,...,...
495,O'Dwyer's PR News,#ChatGPT set off an endless stream of fear and...,"New York, NY","The #1 news outlet for public relations, publi...",2009-02-25 15:50:27+00:00,15013.0,2876.0,1,False,2023-05-17 16:00:37+00:00,['ChatGPT'],Twitter Web App
496,"Tracy Krauss, #author #artist #playwright",ChatGPT is a hot topic; there is no doubt. It ...,"British Columbia, Canada",I write modern Christian romance with a twist ...,2010-06-19 04:45:43+00:00,2718.0,1803.0,348,False,2023-05-17 16:00:37+00:00,NaN,Hootsuite Inc.
497,Tom Solid,Exciting news! #chatGPT has launched its Plugi...,Germany,🚀✨ We help you to build your productivity syst...,2010-12-04 11:00:53+00:00,3460.0,332.0,4464,False,2023-05-17 16:00:30+00:00,['chatGPT'],Hootsuite Inc.
498,Tweeting Monk,@Oncrawl Agreed! The #ChatGPT is definitely th...,GET 30% OFF FOR 3 MONTHS,Set your Twitter account on auto-pilot and wat...,2022-03-30 10:35:13+00:00,50.0,111.0,64,False,2023-05-17 16:00:28+00:00,['ChatGPT'],TweetingMonk


<h1>Preprocessing

In [30]:
#Remove nan values
def drop_nul(data):
    data = data.dropna()
    return data

In [41]:
#Remove duplicates
def remove_duplicates(data):
    data = data.drop_duplicates().reset_index()
    return data

In [42]:
#Normalize
def Normalize(data):
    scaler = MinMaxScaler()
    cols_to_norm = ["user_followers", "user_friends", "user_favourites"]
    data[cols_to_norm] = scaler.fit_transform(data[cols_to_norm])
    return data

In [96]:
#remove stopwords and punctuation
def remove_stopwords_punc(text_list):
    stop_words = set(stopwords.words('english'))
    
    final_tokens = []
    for text in text_list:
        words = word_tokenize(text)
        no_stop = []
        
        for w in words:
            if w not in stop_words and w.isalpha():
                no_stop.append(w)
        
        final_tokens.append(no_stop)
    return final_tokens    

In [97]:
#applying stemming
def apply_stemming(words_list):
    stemmer = PorterStemmer()
    final_stems = [[stemmer.stem(w) for w in text] for text in words_list]
    return final_stems

In [98]:
#applying lematization
def apply_lemmatization(text_list):
    lemmatizer = WordNetLemmatizer()
    final_lem = [[lemmatizer.lemmatize(token) for token in text] for text in text_list]
    return final_lem

In [99]:
df = drop_nul(df2)
df = remove_duplicates(df)
df = Normalize(df)
df["stop_words_punc_removed"] = remove_stopwords_punc(df["text"])
df["stems"] = apply_stemming(df["stop_words_punc_removed"])
df["lems"] = apply_lemmatization(df["stop_words_punc_removed"])

In [100]:
df["stop_words_punc_removed"]

0      [ChatGPT, Thinks, These, Crypto, Coins, Will, ...
1      [New, JWVance, post, startups, including, Vcin...
2      [Out, Binance, Recommendation, Short, Ticker, ...
3      [Video, Show, company, logo, based, website, d...
4      [I, used, CerberusGPT, HologeneticsLab, simula...
                             ...                        
269    [lucky, chatGPT, constantly, makes, errors, er...
270    [Hi, everyone, want, share, blogs, https, Chat...
271    [ChatGPT, set, endless, stream, fear, despair,...
272    [Exciting, news, chatGPT, launched, Plugin, St...
273    [Oncrawl, Agreed, The, ChatGPT, definitely, su...
Name: stop_words_punc_removed, Length: 274, dtype: object